In [35]:
import numpy as np
import pandas as pd
import datetime
from datetime import datetime, date
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 8)
pd.set_option('display.width', 80)

export = pd.read_excel("T:\Ручки\Py\Export9.xlsx",
                  index_col='Материал')
export = export[['Завод', 'Дата проводки', 'Кол-во в ЕИ ввода', 'Вид движения']]
export = export.rename(columns=
              {'Дата проводки': 'Дата',
              'Кол-во в ЕИ ввода': 'Количество',
              'Вид движения': 'Вид'})

u = pd.read_excel("T:\разное\Действительно нужные документы\Прайс-лист РУЧКИ с 01 01 2019.xlsx",
                sheet_name='Union Knopf',
                index_col='Article')
e = pd.read_excel("T:\разное\Действительно нужные документы\Прайс-лист РУЧКИ с 01 01 2019.xlsx",
                sheet_name='Eureka',
                index_col='Article')
g = pd.read_excel("T:\разное\Действительно нужные документы\Прайс-лист РУЧКИ с 01 01 2019.xlsx",
                sheet_name='Giusti',
                index_col='Article')
m = pd.read_excel("T:\разное\Действительно нужные документы\Прайс-лист РУЧКИ с 01 01 2019.xlsx",
                sheet_name='Marella design',
                index_col='Article')
b = pd.read_excel("T:\разное\Действительно нужные документы\Прайс-лист РУЧКИ с 01 01 2019.xlsx",
                sheet_name='Boyard',
                index_col='Article')
v = pd.read_excel("T:\разное\Действительно нужные документы\Прайс-лист РУЧКИ с 01 01 2019.xlsx",
                sheet_name='Валмакс',
                index_col='Article')
c = pd.read_excel("T:\разное\Действительно нужные документы\Прайс-лист РУЧКИ с 01 01 2019.xlsx",
                sheet_name='Cosma',
                index_col='Article')
price = pd.concat([u, e, g, m, b, v, c], sort=True)
price.rename(columns=
        {'Новая цена для Фиеры Трейдинг': 'Цена ФТ'},
        inplace=True)
price = price.round(2)
price = price[['Наименование', 'Себестоимость', 'Цена, закуп', 'Цена ФТ', 'Мин партия', 'Кратность']]

zakaz = pd.read_excel("T:\Ручки\Закупки\Заказанные артикулы. Европа.xlsx",
                  index_col='Артикул',
                    inplace=True)
zakaz = zakaz[['Кол-во']]
zakaz = zakaz.groupby(['Артикул'])
zakaz = zakaz.agg({'Кол-во': np.sum})
zakaz.rename(columns=
        {'Кол-во': 'Заказано'},
        inplace=True)

salony = pd.read_excel("T:\Ручки\Образцы\Таблица образцов в салонах клиентов.xlsx",
                sheet_name='Для программы',
                index_col='Артикул')
salony = salony.reset_index()
salony.rename(columns=
        {'Филиал': 'Завод'},
        inplace=True)
salony.rename(columns=
        {'Артикул': 'Материал'},
        inplace=True)
reindexed = salony.reset_index()
salony = reindexed.set_index(['Материал'])
salony.rename(columns=
        {'Количество': 'В салонах'},
        inplace=True)
salony = salony[['Завод', 'В салонах']]
#export[:5]
#price[:5]
#zakaz
#salony

export_s_vozvratom = export[(export.Вид == 601) | (export.Вид == 602) | (export.Вид == 901)]
export_bez_vozvrata = export[(export.Вид == 601) | (export.Вид == 901)]
export_s_vozvratom_1 = export_s_vozvratom.groupby(['Материал'])

prodano_vsego = export_s_vozvratom_1.agg({'Количество' : sum,
       'Дата' : max})

export_bez_vozvrata = export_bez_vozvrata.groupby(['Материал'])
kolichestvo_prodazh = export_bez_vozvrata.count()
kolichestvo_prodazh = kolichestvo_prodazh.drop(['Завод', 'Дата', 'Вид'], axis = 1)
kolichestvo_prodazh.rename(columns=
        {'Количество': 'Количество_продаж'},
        inplace=True)
kolichestvo_vozvratov = export[(export.Вид == 602)]
kolichestvo_vozvratov = kolichestvo_vozvratov.groupby(['Материал'])
kolichestvo_vozvratov = kolichestvo_vozvratov.count()
kolichestvo_vozvratov = kolichestvo_vozvratov.drop(['Завод', 'Дата', 'Вид'], axis = 1)
kolichestvo_vozvratov.rename(columns=
        {'Количество': 'Количество_возвратов'},
        inplace=True)
kolichestvo_prodazh = pd.concat([kolichestvo_prodazh, kolichestvo_vozvratov], axis=1, sort=True)
kolichestvo_prodazh = kolichestvo_prodazh.fillna(0)
kolichestvo_prodazh['Количество продаж'] = kolichestvo_prodazh['Количество_продаж'] - kolichestvo_prodazh['Количество_возвратов']
kolichestvo_prodazh = kolichestvo_prodazh.drop(['Количество_продаж', 'Количество_возвратов'], axis = 1)
kolichestvo_prodazh

ostatok = export[(export.Завод == 1100) | (export.Завод == 2900)]
ostatok.rename(columns=
       {'Кол-во в ЕИ ввода': 'Количество'},
          inplace=True)
ostatok = ostatok.groupby(['Материал'])
ostatok = ostatok.agg({'Количество': sum,
             'Дата': max})

tab = pd.concat([prodano_vsego, kolichestvo_prodazh], axis=1)
tab.rename(columns=
        {'Дата': 'Дата_посл_прод'},
           inplace=True)
tab.rename(columns=
        {'Количество': 'Продано_шт'},
           inplace=True)
itog = pd.concat([tab, ostatok], axis=1, sort=True)
itog = itog[['Количество', 'Продано_шт', 'Количество продаж', 'Дата_посл_прод', 'Дата']]
itog = itog.fillna(0)

export_s_vozvratom['Time'] = pd.Timestamp('now')

export_s_vozvratom['TimeDelta'] = export_s_vozvratom.Time - export_s_vozvratom.Дата

export_s_vozvratom1 = export_s_vozvratom[export_s_vozvratom.TimeDelta <= '30 days']
export_s_vozvratom1 = export_s_vozvratom1[['Количество']]
export_s_vozvratom1.rename(columns=
        {'Количество': 'Количество1'},
           inplace=True)
export_s_vozvratom1 = export_s_vozvratom1.groupby(['Материал'])
export_s_vozvratom1 = export_s_vozvratom1.agg({'Количество1' : np.sum})

export_s_vozvratom2 = export_s_vozvratom[(export_s_vozvratom.TimeDelta <= '60 days') & (export_s_vozvratom.TimeDelta > '30 days')]
export_s_vozvratom2 = export_s_vozvratom2[['Количество']]
export_s_vozvratom2.rename(columns=
        {'Количество': 'Количество2'},
           inplace=True)
export_s_vozvratom2 = export_s_vozvratom2.groupby(['Материал'])
export_s_vozvratom2 = export_s_vozvratom2.agg({'Количество2' : np.sum})

export_s_vozvratom3 = export_s_vozvratom[(export_s_vozvratom.TimeDelta <= '90 days') & (export_s_vozvratom.TimeDelta > '60 days')]
export_s_vozvratom3 = export_s_vozvratom3[['Количество']]
export_s_vozvratom3.rename(columns=
        {'Количество': 'Количество3'},
           inplace=True)
export_s_vozvratom3 = export_s_vozvratom3.groupby(['Материал'])
export_s_vozvratom3 = export_s_vozvratom3.agg({'Количество3' : np.sum})

export_s_vozvratom4 = export_s_vozvratom[(export_s_vozvratom.TimeDelta <= '120 days') & (export_s_vozvratom.TimeDelta > '90 days')]
export_s_vozvratom4 = export_s_vozvratom4[['Количество']]
export_s_vozvratom4.rename(columns=
        {'Количество': 'Количество4'},
           inplace=True)
export_s_vozvratom4 = export_s_vozvratom4.groupby(['Материал'])
export_s_vozvratom4 = export_s_vozvratom4.agg({'Количество4' : np.sum})

export_s_vozvratom5 = export_s_vozvratom[(export_s_vozvratom.TimeDelta <= '150 days') & (export_s_vozvratom.TimeDelta > '120 days')]
export_s_vozvratom5 = export_s_vozvratom5[['Количество']]
export_s_vozvratom5.rename(columns=
        {'Количество': 'Количество5'},
           inplace=True)
export_s_vozvratom5 = export_s_vozvratom5.groupby(['Материал'])
export_s_vozvratom5 = export_s_vozvratom5.agg({'Количество5' : np.sum})

export_s_vozvratom6 = export_s_vozvratom[(export_s_vozvratom.TimeDelta <= '180 days') & (export_s_vozvratom.TimeDelta > '150 days')]
export_s_vozvratom6 = export_s_vozvratom6[['Количество']]
export_s_vozvratom6.rename(columns=
        {'Количество': 'Количество6'},
           inplace=True)
export_s_vozvratom6 = export_s_vozvratom6.groupby(['Материал'])
export_s_vozvratom6 = export_s_vozvratom6.agg({'Количество6' : np.sum})


itog = pd.concat([itog, export_s_vozvratom1, export_s_vozvratom2, export_s_vozvratom3, export_s_vozvratom4, export_s_vozvratom5, export_s_vozvratom6], axis=1, sort=True)
itog = itog.fillna(0)

itog35 = itog[(itog.Количество1 < -5) & (itog.Количество2 < -5) & (itog.Количество3 < -5)]
itog35['3х5'] = 'Да'
itog35 = itog35['3х5']
itog30 = itog[(itog.Количество1 < -30)]
itog30['>30'] = 'Да'
itog30 = itog30['>30']
itog_sum = pd.concat([itog35, itog30], axis=1, sort=True)

salony = salony.reset_index()
salony.rename(columns=
        {'Артикул': 'Материал'},
        inplace=True)
reindexed = salony.reset_index()
salony = reindexed.set_index(['Материал'])

salony.rename(columns=
        {'Количество': 'В салонах'},
        inplace=True)

salony = salony.join(prodano_vsego, how='inner')

salony = salony[['В салонах']]

salony = salony.groupby(level=['Материал'])
salony = salony.agg({'В салонах': np.sum})
itog_sum = pd.concat([itog_sum, salony], axis=1, sort=True)

raspr = itog[['Количество1', 'Количество2', 'Количество3', 'Количество4', 'Количество5', 'Количество6']]
raspr = raspr * (-1)
raspr = raspr.fillna(0)
raspr['Продано за 3 посл мес'] = raspr['Количество1'] + raspr['Количество2'] + raspr['Количество3']
#raspr['Продано за 3 пред мес'] = raspr['Количество4'] + raspr['Количество5'] + raspr['Количество6'] #+ raspr['Количество1'] + raspr['Количество2'] + raspr['Количество3']
raspr = raspr[['Продано за 3 посл мес']]#, 'Продано за 3 пред мес']]
#raspr1 = raspr[['Продано за 3 посл мес']]
raspr = raspr[raspr['Продано за 3 посл мес'] > 0]
raspr = raspr.sort_values(by = ['Продано за 3 посл мес'], ascending = False)
raspr['Сумма'] = raspr['Продано за 3 посл мес'].sum()
raspr['Процент'] = raspr['Продано за 3 посл мес'] / raspr['Сумма'] * 100
raspr['Процент'] = round(raspr['Процент'], 2)
raspr = raspr[['Продано за 3 посл мес', 'Процент']]

percent_count = 0
index = 0
for i, row in raspr.iterrows():
    percent_count += row['Процент']
    if percent_count > 90:
        index = i
        break
raspr1 = raspr[:index]
raspr1 = raspr1['Процент']

itog_sum = pd.concat([itog_sum, raspr1], axis=1, sort=True)
itog_sum = pd.concat([itog_sum, itog], axis=1, sort=True)
itog_sum.dropna(thresh=12,axis=0, inplace=True)

itog_sum['Прогноз'] = round((itog_sum.Количество1*0.4 + itog_sum.Количество2*0.3 + itog_sum.Количество3*0.2 + 
itog_sum.Количество4*0.05 + itog_sum.Количество5*0.05)*3*1.52*(-1))
#=======================================================================================================#
itog_sum['Необходимо_заказать'] = round(itog_sum.Прогноз - itog_sum.Количество)

itog_sum = itog_sum[['3х5','>30', 'В салонах', 'Процент', 'Количество', 'Продано_шт', 'Количество продаж', 
                    'Прогноз', 'Необходимо_заказать']]
itog_sum = pd.concat([itog_sum, price], axis=1, sort=True)
itog_sum.dropna(thresh=6,axis=0, inplace=True)
itog_sum = pd.concat([itog_sum, zakaz], axis=1, sort=True)
itog_sum.dropna(thresh=10,axis=0, inplace=True)
itog_sum['Заказано'] = itog_sum['Заказано'].fillna(0)
itog_sum['Заказ'] = round((itog_sum.Необходимо_заказать - itog_sum['Заказано']) / itog_sum['Кратность'] + 0.5) * itog_sum['Кратность']

itog_sum.loc[itog_sum['Заказ'] < 0, 'Заказ'] = 0
itog_sum_1 = itog_sum[itog_sum['Заказ'] != 0]
itog_sum_1.loc[itog_sum_1['Заказ'] < itog_sum_1['Мин партия'], 'Заказ'] = itog_sum_1['Мин партия']
itog_sum_2 = itog_sum[itog_sum['Заказ'] == 0]
itog_sum2 = pd.concat([itog_sum_1, itog_sum_2])
itog_sum2['Стоимость'] = itog_sum2.Заказ * itog_sum2['Цена, закуп']
itog_sum2 = itog_sum2.sort_values(['Заказ'], ascending=False)
itog_sum2 = itog_sum2[['Наименование', '3х5', '>30', 'В салонах', 'Процент', 'Количество', 'Продано_шт', 'Количество продаж', 
                    'Прогноз', 'Необходимо_заказать', 'Заказано', 'Заказ', 'Стоимость']]

itog1 = itog[itog['Количество'] > 0]
itog1 = itog1.fillna(0)
itog1['Time_now'] = pd.Timestamp('now')
itog1['TimeDeliver'] = itog1.Time_now - itog1.Дата
ost = itog1[(itog1.Количество1 == 0) & (itog1.Количество2 == 0) & (itog1.Количество3 == 0) & (itog1.TimeDeliver > '90 days')]
#ost = tab123q[((tab123q.Количество1 + tab123q.Количество2 + tab123q.Количество3) > (0)) & (tab123q.TimeDeliver > '90 days')]
ost1 = itog1[(itog1.Количество1 == 0) & (itog1.Количество2 == 0) & (itog1.Количество3 == 0) & (itog1.Количество4 == 0) & (itog1.Количество5 == 0) & (itog1.Количество6 == 0) & (itog1.TimeDeliver > '180 days')]
#ost1 = tab123q[((tab123q.Количество1 + tab123q.Количество2 + tab123q.Количество3 + tab123q.Количество4 + tab123q.Количество5 + tab123q.Количество6) > (0)) & (tab123q.TimeDeliver > '180 days')]
ost = ost[['Количество', 'Продано_шт', 'Количество продаж', 'Дата_посл_прод', 'Дата']]
ost1 = ost1[['Количество', 'Продано_шт', 'Количество продаж', 'Дата_посл_прод', 'Дата']]
ost = ost.sort_values(['Количество'], ascending=False)
ost1 = ost1.sort_values(['Количество'], ascending=False)
malo = itog[itog.Количество < 30]
malo = malo[['Количество', 'Продано_шт', 'Количество продаж']]
malo = pd.concat([malo, zakaz], axis=1, sort=True)
malo.dropna(thresh=3,axis=0, inplace=True)

from pandas import ExcelWriter
with ExcelWriter ("T:\Ручки\Py\itog_sum.xlsx") as writer:
    itog_sum2.to_excel(writer, sheet_name='Заказ')
    ost.to_excel(writer, sheet_name='Не продаётся 3 мес!')
    ost1.to_excel(writer, sheet_name='Не продаётся 6 мес!')
    malo.to_excel(writer, sheet_name='Мало на складе')

#itog_sum



C:\Users\goryunov\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\goryunov\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\goryunov\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in